# beamlet utilities

> API details.

In [ ]:
#default_exp util

In [ ]:
#export 

from numpy.fft import fftfreq
import numpy as np
from scipy.spatial import ConvexHull, Delaunay
from metpy.interpolate import geometry
from tqdm import trange


In [ ]:
#export 
def natural_neighbor_weights_internal(xp, yp, grid_loc, tri, neighbors, circumcenters, minimum_weight_cutoff=1e-2):
    r"""Generate a natural neighbor interpolation of the observations to the given point.

    This uses the Liang and Hale approach [Liang2010]_. The interpolation will fail if
    the grid point has no natural neighbors.

    Parameters
    ----------
    xp: (N, ) ndarray
        x-coordinates of observations
    yp: (N, ) ndarray
        y-coordinates of observations
    variable: (N, ) ndarray
        observation values associated with (xp, yp) pairs.
        IE, variable[i] is a unique observation at (xp[i], yp[i])
    grid_loc: (float, float)
        Coordinates of the grid point at which to calculate the
        interpolation.
    tri: `scipy.spatial.Delaunay`
        Delaunay triangulation of the observations.
    neighbors: (N, ) ndarray
        Simplex codes of the grid point's natural neighbors. The codes
        will correspond to codes in the triangulation.
    circumcenters: list
        Pre-calculated triangle circumcenters for quick look ups. Requires
        indices for the list to match the simplices from the Delaunay triangulation.
    minimum_weight_cutoff:
        delete weights smaller than minimum_weight_cutoff
    Returns
    -------
    weights: (W,) ndarray
       weights for interpolating the grid location
    point_indices  (W,) ndarray
        integer indices of the points used for interpolation
    """
    edges = geometry.find_local_boundary(tri, neighbors)
    edge_vertices = [segment[0] for segment in geometry.order_edges(edges)]
    num_vertices = len(edge_vertices)

    p1 = edge_vertices[0]
    p2 = edge_vertices[1]

    c1 = geometry.circumcenter(grid_loc, tri.points[p1], tri.points[p2])
    polygon = [c1]

    area_list = []
    pt_list = []
    total_area = 0.0
    indices = np.arange(xp.shape[0])
    for i in range(num_vertices):

        p3 = edge_vertices[(i + 2) % num_vertices]

        try:
            c2 = geometry.circumcenter(grid_loc, tri.points[p3], tri.points[p2])
            polygon.append(c2)

            for check_tri in neighbors:
                if p2 in tri.simplices[check_tri]:
                    polygon.append(circumcenters[check_tri])
            # print('polygon',polygon)
            pts1 = [polygon[i] for i in ConvexHull(polygon).vertices]
            pt_mask = (tri.points[p2][0] == xp) & (tri.points[p2][1] == yp)
            pt_list.append(indices[pt_mask][0])
            cur_area = geometry.area(pts1)

            total_area += cur_area

            area_list.append(cur_area)

        except (ZeroDivisionError) as e:
            message = ('Error during processing of a grid. '
                       'Interpolation will continue but be mindful '
                       f'of errors in output. {e}')

            print(message)
            return np.nan

        polygon = [c2]

        p2 = p3

    weights = np.array([x / total_area for x in area_list])
    pt_list = np.array(pt_list)

    weights1 = weights[weights > minimum_weight_cutoff]
    pt_list1 = pt_list[weights > minimum_weight_cutoff]

    weights1 *= 1 / np.sum(weights1)

    return weights1, pt_list1

In [ ]:
#export 
def natural_neighbor_weights(known_points, interp_points, minimum_weight_cutoff=1e-2):
    r"""Generate a natural neighbor interpolation of the observations to the given points.

    This uses the Liang and Hale approach [Liang2010]_. The interpolation will fail if
    the grid point has no natural neighbors.

    Parameters
    ----------
    known_points: (B_S, 2) ndarray
        x,y-coordinates of observations
    interp_points: (B, 2) ndarray
        Coordinates of the grid point at which to calculate the
        interpolation.
    minimum_weight_cutoff:
        delete weights smaller than minimum_weight_cutoff, default 1e-2
    Returns
    -------
    weights: (B,B_S) ndarray
       weights for interpolating the grid location from the B_S sampled points
    """
    interp_points = interp_points.copy() - 1e-3
    weights = np.zeros((interp_points.shape[0], known_points.shape[0]))

    tri = Delaunay(known_points)

    # members is dict with length B

    # print('interp_points', interp_points)
    for b in trange(interp_points.shape[0], desc="Natural neighbor interpolation"):
        interp_point = interp_points[b]
        interp_point2 = interp_points[[b]]
        radius_subtracted = 0
        while radius_subtracted < 10:
            r = np.linalg.norm(interp_point)
            phi = np.arctan2(interp_point[0], interp_point[1])
            r_new = r - radius_subtracted
            interp_point_new = np.array([r_new * np.sin(phi), r_new * np.cos(phi)])
            interp_point2[:] = interp_point_new
            try:
                # print('interp_point_new', interp_point_new)
                # print('interp_point', interp_point)
                members, circumcenters = geometry.find_natural_neighbors(tri, interp_point2)
                neighbors = members[0]
                # print('members', members)
                # print('b', b)
                # print('neighbors', neighbors)
                # print('known_points[:, 0]', known_points[:, 0])
                # print('known_points[:, 1]', known_points[:, 1])
                #
                # print('circumcenters', circumcenters)
                weights_ind, indices_ind = natural_neighbor_weights_internal(known_points[:, 0], known_points[:, 1],
                                                                             interp_point_new, tri, neighbors,
                                                                             circumcenters,
                                                                             minimum_weight_cutoff=minimum_weight_cutoff)
                break
            except:
                radius_subtracted += 0.1

        weights[b, indices_ind] = weights_ind
    return weights

In [ ]:
#export 
def beamlet_samples(A, radius, n_angular_samples, n_radial_samples):
    """
    Determines which beams of the scattering matrix should be sampled, given an illumination aperture A,
    an aperture radius, the number of angular samples, and the number of radial samples

    :param A:
    :param radius:
    :param n_angular_samples:
    :param n_radial_samples:
    :return: beamlet samples, (Bs,2)
    """
    M = A.shape[0]

    my1 = np.tile(fftfreq(M, d=1 / M)[:, None], M).astype(np.int32)
    mx1 = np.repeat(fftfreq(M, d=1 / M)[:, None].T, M, axis=0).astype(np.int32)

    B = np.nonzero(A)
    beam_coords1 = np.array([my1[B], mx1[B]]).T
    beam_coords2 = beam_coords1 + 1e-2

    a_offset = np.pi / n_angular_samples
    my1 = np.tile(fftfreq(M, d=1 / M)[:, None], M).astype(np.int32)
    mx1 = np.repeat(fftfreq(M, d=1 / M)[:, None].T, M, axis=0).astype(np.int32)
    beam_coords1 = np.array([my1[B], mx1[B]]).T
    radial_samples = np.linspace(0, radius, n_radial_samples, endpoint=True)[1:]
    samples = [[0, 0]]
    for i, r in enumerate(radial_samples):
        angular_samples = np.linspace(-np.pi, np.pi, n_angular_samples * (1 + i), endpoint=False)
        # print(i, len(angular_samples))
        for a in angular_samples:
            # print(r, a)
            # print([r * np.sin(a + a_offset * i), r * np.cos(a + a_offset * i)])
            samples.append([r * np.sin(a + a_offset * i), r * np.cos(a + a_offset * i)])
    samples = np.array(samples)
    xy_diff = np.linalg.norm(samples[None, ...] - beam_coords2[:, None, :], axis=2)
    r_min_indices = np.argmin(xy_diff, axis=0)
    best_beam_coords = beam_coords1[r_min_indices]
    return best_beam_coords